## Constructing a Minimum-Variance Portfolio

First, we need to import all of the libraries that we'll be using:

* NumPy
* pandas
* yfinance
* optimize from scipy

In [ ]:
# Imports
import numpy as np
import pandas as pd
import yfinance as yf
from scipy import optimize

### Collecting and Preparing the Data

In [ ]:
tickers = [
    'AAPL',
    'MSFT',
    'GOOG',
    'AMZN'
]

period = '1y'

interval = '1d'

fields = []
for i in range(len(tickers)):
    fields.append((tickers[i], 'Close'))

data = yf.download(
    tickers = tickers,
    period = period,
    interval = interval,
    group_by = 'ticker'
)[fields]

print(data.head())

Returns

In [ ]:
returns = data.pct_change().dropna() * 100


print(returns.head())

Covariance Matrix

In [ ]:
covariance = returns.cov().to_numpy()

print(covariance)

### Optimization

Objective Function and constraint

In [ ]:
def portfolioVar(w, cov):
    w = np.matrix(w)
    cov = np.matrix(cov)
    result = w * cov * w.T
    return result

constraint = ({'type': 'eq', 'fun': lambda x:  np.sum(x) - 1})

bounds = tuple((-1, 1) for i in tickers)

Initial guess, bounds, and optimal portfolio

In [ ]:

w0 = [1 / len(tickers)] * len(tickers)


result = optimize.minimize(
    fun = portfolioVar,
    x0 = w0,
    args = covariance,
    method = 'SLSQP',
    bounds = bounds,
    constraints = constraint
)

result.x = result.x.round(3)

for i in range(len(tickers)):
    print(tickers[i], '=', result.x[i])